In [2]:
import os
os.environ['PYSPARK_SUBMIT_ARGS'] = '--packages org.apache.spark:spark-streaming-kafka-0-8_2.11:2.3.0 pyspark-shell'


import sys
import time
import json
from pprint import pprint
from pymongo import MongoClient
from pyspark import SparkContext, SparkConf
from pyspark.streaming import StreamingContext
from pyspark.streaming.kafka import KafkaUtils

#function to record data and return as a dict
def storeClimate(climate):
    jsonData={}
    jsonData["latitude"] = climate['data']['latitude']
    jsonData["longitude"] = climate['data']['longitude']
    jsonData["relative_humidity"] = climate['data']['relative_humidity']
    jsonData["windspeed"] = climate['data']['windspeed']
    jsonData["max_windspeed"] = climate['data']['max_windspeed']
    jsonData["air_temperature_celcius"] = climate['data']['air_temperature_celcius']
    jsonData["precipitation"] = climate['data']['precipitation']
    jsonData["created_time"] = climate['data']["created_time"]
    
    return(jsonData)
  
#function to record data and return as a dict
def storeClimateHotspot(climate,hotspot):
    jsonData={}
    jsonData["latitude"] = climate['data']['latitude']
    jsonData["longitude"] = climate['data']['longitude']
    jsonData["relative_humidity"] = climate['data']['relative_humidity']
    jsonData["windspeed"] = climate['data']['windspeed']
    jsonData["max_windspeed"] = climate['data']['max_windspeed']
    jsonData["air_temperature_celcius"] = climate['data']['air_temperature_celcius']
    jsonData["precipitation"] = climate['data']['precipitation']
    jsonData["created_time"] = climate['data']["created_time"]
    jsonData["hotspots"] = hotspot['data']
    
    return(jsonData)

    


def sendDataToDB(iter):
    
    client = MongoClient()
    db = client.fit5148_db
    climate_hist = db.climate_hist 
    terra = {} #empty dict for hotspot terra record
    aqua = {} #empty dict for hotspot aqua record
    climates =[] #empty list for climate records (as there may be more than one record)
    for record in iter:
            data = json.loads(record[1])
            if data['sender_id'] == "CLIMATE":
                climates.append(data)
                #jsonData = {}
                #jsonData["sender_id"] = data['sender_id'] 
                
            elif data['sender_id'] == "AQUA":
                aqua = data
                
            elif data['sender_id'] == "TERRA":
                terra = data
                

            
    for climate in climates: #for every dict in climate
        
        if not aqua: #if there is no aqua data
                    
            if not terra: #if there is no terra data
                jsonData=storeClimate(climate)   
                        
            else: #if there is terra data   
                        
                if climate['geohash'] == terra['geohash']: #if the geohash for climate and terra match
                    jsonData=storeClimateHotspot(climate,terra)   
                            
                else: #if the geohash for climate and terra do not match
                    jsonData=storeClimate(climate)
                            
        else: #if there is aqua data
                    
            if not terra: #if there is no terra data
                        
                if climate['geohash'] == aqua['geohash']: #if the geohash for climate and aqua match
                    jsonData=storeClimateHotspot(climate,aqua)
                            
                else: #if they do not match
                    jsonData=storeClimate(climate)
                            
            else: #if there is terra data
                        
                if aqua['geohash'] == terra['geohash']: #if the geohash for aqua and terra match
                            
                    if aqua['geohash'] == climate['geohash']: #if the geohash for aqua, terra and climate match
                        hotspot={}
                        hotspot=aqua
                        hotspot["surface_temperature_celcius"] = str((int(aqua["surface_temperature_celcius"])+int(terra["surface_temperature_celcius"]))/2)
                        hotspot["confidence"] = str((int(aqua["confidence"])+int(terra["confidence"]))/2)
                        jsonData=storeClimateHotspot(climate,hotspot)
                                
                    else: #if there is no match, only store climate data
                        jsonData=storeClimate(climate)
                        
                #if geo for aqua and terra do not match, but aqua matches with climate
                elif aqua['geohash'] == climate['geohash']:
                    jsonData=storeClimateHotspot(climate,aqua)
                        
                #if geo for aqua and terra do not match, but terra matches with climate
                elif terra['geohash'] == climate['geohash']:
                    jsonData=storeClimateHotspot(climate,terra)
                        
                #if none of the geohashes match, only store climate data    
                else:
                    jsonData=storeClimate(climate)        
        try:
              climate_hist.insert(jsonData)
        except Exception as ex:
             print("Exception Occured. Message: {0}".format(str(ex)))
                
    client.close()

    
n_secs = 10 #batch interval is 10 seconds
topic = "assignment"

conf = SparkConf().setAppName("KafkaStreamProcessor").setMaster("local[2]") # two execution threads
sc = SparkContext.getOrCreate()
if sc is None:
    sc = SparkContext(conf=conf)
sc.setLogLevel("WARN")
ssc = StreamingContext(sc, n_secs)
    
climateStream = KafkaUtils.createDirectStream(ssc, [topic], {
                        'bootstrap.servers':'127.0.0.1:9092', 
                        'group.id':'climate', 
                        'fetch.message.max.bytes':'15728640',
                        'auto.offset.reset':'largest'})
                        # Group ID is completely arbitrary

print("Data being processed")
climate = climateStream.foreachRDD(lambda rdd: rdd.foreachPartition(sendDataToDB))

climateStream.pprint() #just a debugger to see what is being processed

print("Start")
ssc.start()
print("Sleep")
time.sleep(1200) # Run stream for 20 minutes just in case no detection of producer
#ssc.awaitTermination()
ssc.stop(stopSparkContext=True,stopGraceFully=True)
print("end")

Data being processed
Start
Sleep
-------------------------------------------
Time: 2019-05-24 10:43:50
-------------------------------------------
('jsondata', '{"geohash": "r365q", "data": {"relative_humidity": "53.6", "max_windspeed": "15", "air_temperature_celcius": "18", "precipitation": " 0.00I", "created_time": "10:43:42", "latitude": "-37.382", "longitude": "149.341", "windspeed": "7.2"}, "sender_id": "CLIMATE"}')
('jsondata', '{"geohash": "r336f", "data": {"relative_humidity": "50.7", "max_windspeed": "13", "air_temperature_celcius": "14", "precipitation": " 0.12G", "created_time": "10:43:47", "latitude": "-37.461", "longitude": "148.109", "windspeed": "8.6"}, "sender_id": "CLIMATE"}')

-------------------------------------------
Time: 2019-05-24 10:44:00
-------------------------------------------
('jsondata', '{"geohash": "r337d", "data": {"relative_humidity": "35.6", "max_windspeed": "20", "air_temperature_celcius": "10", "precipitation": " 0.00I", "created_time": "10:43:52"

KeyboardInterrupt: 

-------------------------------------------
Time: 2019-05-24 10:45:30
-------------------------------------------
('jsondata', '{"geohash": "r1q24", "data": {"relative_humidity": "59.7", "max_windspeed": "15.9", "air_temperature_celcius": "20", "precipitation": " 0.01G", "created_time": "10:45:22", "latitude": "-37.954", "longitude": "143.918", "windspeed": "8.6"}, "sender_id": "CLIMATE"}')

-------------------------------------------
Time: 2019-05-24 10:45:40
-------------------------------------------

-------------------------------------------
Time: 2019-05-24 10:45:50
-------------------------------------------

-------------------------------------------
Time: 2019-05-24 10:46:00
-------------------------------------------

-------------------------------------------
Time: 2019-05-24 10:46:10
-------------------------------------------

-------------------------------------------
Time: 2019-05-24 10:46:20
-------------------------------------------

------------------------------